In [27]:
import keras
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Flatten, Embedding, regularizers
from keras.layers import Conv1D, MaxPooling1D
from keras.layers.merge import Concatenate
from keras import optimizers
from keras.preprocessing import sequence
from keras import backend as K
from w2v import train_word2vec 
from keras.utils import np_utils
import pickle, datetime
import numpy as np
import pandas as pd
from scipy.signal import convolve2d

from gensim import corpora
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import SnowballStemmer
from w2v import train_word2vec 
import pickle, datetime
import difflib

## Loading Data

In [286]:
sentiment_test

array([2, 3, 4, ..., 4, 3, 0])

In [398]:
df = pd.read_csv('../../Datasets/SST1_dataset/Processed_SST1.tsv', sep='\t')
train_df = pd.read_csv('../../Datasets/Kaggle_dataset/train.tsv', sep='\t', header=0)

raw_docs_train      = df[df.split_ind == 1]['Phrases'].values
sentiment_train     = df[df.split_ind == 1]['Label'].values
raw_docs_test       = df[df.split_ind == 2]['Phrases'].values
sentiment_test      = df[df.split_ind == 2]['Label'].values
raw_docs_test       = np.append(raw_docs_test, ['I hate this movie'])
sentiment_test      = np.append(sentiment_test, [1])
num_labels          = len(np.unique(sentiment_train))

N_TRAIN = len(raw_docs_train)
N_TEST = len(raw_docs_test)

In [399]:
#text pre-processing
stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", ':', ';', '(', ')', '[', ']', '{', '}'])
stemmer = SnowballStemmer('english')

print ("pre-processing train docs...")
processed_docs_train = []
for doc in raw_docs_train:
   tokens = word_tokenize(doc)
   filtered = [word for word in tokens if word not in stop_words]
   stemmed = [stemmer.stem(word) for word in filtered]
   processed_docs_train.append(stemmed)

print ("pre-processing test docs...")
processed_docs_test = []
for doc in raw_docs_test:
   tokens = word_tokenize(doc)
   filtered = [word for word in tokens if word not in stop_words]
   stemmed = [stemmer.stem(word) for word in filtered]
   processed_docs_test.append(stemmed)
    
processed_docs_all = np.concatenate((processed_docs_train, processed_docs_test), axis=0)

dictionary = corpora.Dictionary(processed_docs_all)
dictionary_size = len(dictionary.keys())

print ("converting to token ids...")
word_id_train, word_id_len = [], []
for doc in processed_docs_train:
    word_ids = [dictionary.token2id[word] for word in doc]
    word_id_train.append(word_ids)
    word_id_len.append(len(word_ids))

word_id_test, word_ids = [], []
for doc in processed_docs_test:
    word_ids = [dictionary.token2id[word] for word in doc]
    word_id_test.append(word_ids)
    word_id_len.append(len(word_ids))
        
seq_len = np.round((np.mean(word_id_len) + 2*np.std(word_id_len))).astype(int)

#pad sequences
x_train = sequence.pad_sequences(np.array(word_id_train), maxlen=seq_len)
x_test  = sequence.pad_sequences(np.array(word_id_test), maxlen=seq_len)
y_train = np_utils.to_categorical(sentiment_train, num_labels)
y_test  = np_utils.to_categorical(sentiment_test, num_labels)

pre-processing train docs...
pre-processing test docs...
converting to token ids...


In [462]:
filename = '/home/shikhar/Dropbox/IISC/Machine_Learning/ML_Project/Images/sst_cnn_rand/sst_cnn_rand_3'

out = []
with open( filename, 'rb') as input:
    out = pickle.load(input)
    
model = Model.from_config(out[0])
model.set_weights(out[1])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_9 (InputLayer)             (None, 21)            0                                            
____________________________________________________________________________________________________
embedding (Embedding)            (None, 21, 100)       1380100                                      
____________________________________________________________________________________________________
dropout_12 (Dropout)             (None, 21, 100)       0                                            
____________________________________________________________________________________________________
conv1d_16 (Conv1D)               (None, 19, 50)        15050                                        
___________________________________________________________________________________________

In [463]:
model.compile(loss="categorical_crossentropy", optimizer=optimizers.SGD(0.01), metrics=["accuracy"])

# Final evaluation of the model
scores = model.evaluate(x_train, y_train, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 33.77%
Accuracy: 28.18%


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_9 (InputLayer)             (None, 21)            0                                            
____________________________________________________________________________________________________
embedding (Embedding)            (None, 21, 100)       1380100                                      
____________________________________________________________________________________________________
dropout_12 (Dropout)             (None, 21, 100)       0                                            
____________________________________________________________________________________________________
conv1d_16 (Conv1D)               (None, 19, 50)        15050                                        
___________________________________________________________________________________________

### Occlusion Method

In [403]:
vec = word_id_test[0]
x_test1 = sequence.pad_sequences([vec], maxlen=seq_len)

In [404]:
dictionary.token2id
dictionary.id2token = dict((v, k) for k, v in dictionary.token2id.items())

In [405]:
test_pred = model.predict(x_test)

In [406]:
test_pred = np.argmax(test_pred, axis=1)
y_actual = np.argmax(y_test, axis=1)

In [407]:
print(np.sum(test_pred == y_actual)*1.0 / y_actual.shape[0])

0.304387155133


In [408]:
crct_pred = (test_pred == y_actual)
ind = []
for i in range(crct_pred.shape[0]):
    if crct_pred[i] == 1:
        ind.append(i)

In [409]:
y_actual

array([2, 3, 4, ..., 3, 0, 1])

In [410]:
k = -1
print(y_actual[ind[k]])
vec = word_id_test[ind[k]]

org_pred =  model.predict(x_test[ind[k]:ind[k]+1, :])
print('Original Prediction', org_pred)

for ele in list(set(vec)):
    vec_temp = [i for i in vec if i != ele]    
    x_test1 = sequence.pad_sequences([vec_temp], maxlen=seq_len, padding="post", truncating="post")
    pred = model.predict(x_test1)
    print(dictionary.id2token[ele])
    print(pred)
#     print(dictionary.id2token[ele], '\t', pred )


1
Original Prediction [[ 0.15741763  0.27602369  0.24537784  0.21322405  0.10795683]]
i
[[ 0.1766758   0.3095786   0.24850512  0.17698482  0.08825567]]
movi
[[ 0.17958815  0.31474501  0.24892661  0.17189257  0.08484766]]
hate
[[ 0.17677309  0.31203997  0.249431    0.17556125  0.08619471]]


In [411]:
counter = 0
for layer in model.layers:
    print('\nFor layer %d -----------------' % counter)
    print(layer.get_config())
    print('Input shape', layer.input_shape)
    print('Ouput shape', layer.output_shape)
#     print(layer.compute_output_shape())
    counter += 1


For layer 0 -----------------
{'batch_input_shape': (None, 21), 'sparse': False, 'dtype': 'float32', 'name': 'input_9'}
Input shape (None, 21)
Ouput shape (None, 21)

For layer 1 -----------------
{'input_length': 21, 'embeddings_regularizer': None, 'dtype': 'int32', 'batch_input_shape': (None, 21), 'trainable': True, 'embeddings_constraint': None, 'mask_zero': False, 'output_dim': 100, 'name': 'embedding', 'embeddings_initializer': {'class_name': 'RandomUniform', 'config': {'seed': None, 'minval': -0.05, 'maxval': 0.05}}, 'activity_regularizer': None, 'input_dim': 13801}
Input shape (None, 21)
Ouput shape (None, 21, 100)

For layer 2 -----------------
{'trainable': True, 'rate': 0.5, 'name': 'dropout_12'}
Input shape (None, 21, 100)
Ouput shape (None, 21, 100)

For layer 3 -----------------
{'bias_constraint': None, 'kernel_size': (3,), 'padding': 'valid', 'kernel_initializer': {'class_name': 'VarianceScaling', 'config': {'distribution': 'uniform', 'seed': None, 'mode': 'fan_avg', 's

In [422]:
# Final Res
y_out = np.eye(5)
for i in range(len(out[1])):
    print(out[1][i].shape)

(13801, 100)
(3, 100, 50)
(50,)
(4, 100, 50)
(50,)
(5, 100, 50)
(50,)
(150, 50)
(50,)
(50, 5)
(5,)


In [423]:
len(raw_docs_test)

2211

In [424]:
test_pred

array([1, 3, 1, ..., 1, 1, 1])

In [425]:
raw_docs_test[2210]

'I hate this movie'

In [426]:
ind

[1,
 8,
 10,
 11,
 14,
 17,
 24,
 25,
 26,
 28,
 32,
 33,
 34,
 36,
 37,
 44,
 50,
 51,
 56,
 58,
 61,
 62,
 64,
 74,
 75,
 77,
 78,
 82,
 83,
 88,
 89,
 91,
 100,
 106,
 109,
 113,
 118,
 120,
 128,
 130,
 133,
 140,
 141,
 142,
 143,
 145,
 148,
 149,
 150,
 152,
 153,
 160,
 162,
 163,
 165,
 169,
 170,
 175,
 178,
 183,
 184,
 185,
 187,
 191,
 193,
 198,
 199,
 201,
 205,
 207,
 208,
 209,
 210,
 214,
 217,
 219,
 222,
 225,
 227,
 228,
 231,
 235,
 245,
 246,
 249,
 252,
 255,
 256,
 260,
 267,
 275,
 280,
 285,
 291,
 296,
 298,
 307,
 314,
 316,
 318,
 320,
 327,
 331,
 335,
 337,
 339,
 346,
 349,
 350,
 353,
 358,
 375,
 382,
 383,
 391,
 392,
 397,
 399,
 406,
 414,
 415,
 416,
 418,
 419,
 424,
 426,
 428,
 430,
 439,
 442,
 444,
 452,
 457,
 459,
 463,
 468,
 473,
 475,
 477,
 479,
 481,
 483,
 484,
 485,
 489,
 490,
 501,
 505,
 507,
 510,
 512,
 517,
 524,
 525,
 526,
 533,
 535,
 537,
 546,
 547,
 553,
 554,
 555,
 556,
 559,
 562,
 563,
 564,
 565,
 567,
 574,
 577,
 5

In [442]:
embed_wt = out[1][0]
# 0 456 134 CNN rand
k = 0
list_ind = word_id_test[ind[k]]
x_input = embed_wt[list_ind, :]
y_final = y_out[:,y_actual[ind[k]]]
y_final

array([ 0.,  0.,  0.,  1.,  0.])

In [443]:
w1 = out[1][9]
y1 = w1 @ y_final
y1 = y1 * (y1 > 0)

w2 = out[1][7]
y2 = w2 @ y1
y2 = y2 * (y2 > 0) # Guided backward pass: backprop
print(y2.shape)

z1 = y2[0:50,]
z2 = y2[50:100,]
z3 = y2[100:150,]

W1 = out[1][1]
W2 = out[1][3]
W3 = out[1][5]
print(W1.shape, W2.shape, W3.shape)

x_viz = x_input


o1 = np.empty((50, ), np.float32)

i1 = np.empty((50, ), np.int32)
for i in range(50):
    output = convolve2d(x_viz, W1[:,:,i], mode='valid')
    output = output * (output > 0) 
    o1[i] = np.max(output, axis = 0)
    i1[i] = np.argmax(output, axis = 0)
    
o2 = np.empty((50, ), np.float32)
i2 = np.empty((50, ), np.int32)
for i in range(50):
    output = convolve2d(x_viz, W2[:,:,i], mode='valid')
    output = output * (output > 0) 
    o2[i] = np.max(output, axis=0)
    i2[i] = np.argmax(output, axis=0)
    
o3 = np.empty((50, ), np.float32)
i3 = np.empty((50, ), np.int32)
for i in range(50):
    output = convolve2d(x_viz, W3[:,:,i], mode='valid')
    output = output * (output > 0) 
    o3[i] = np.max(output, axis=0)
    i3[i] = np.argmax(output, axis=0)
    
print(o1.shape, o2.shape, o3.shape)

# Backward pass guided backpropogation (backprop)
t1 = o1 * z1
t2 = o2 * z2
t3 = o3 * z3

# Backward pass guided backpropogation
# t1 = z1
# t2 = z2
# t3 = z3

# t1 = o1 
# t2 = o2 
# t3 = o3 

tsum = np.sum(t1) + np.sum(t2) + np.sum(t3)

c = np.zeros((21,), np.float32)


for i in range(50):
    for j in range(3):
        c[i1[i] + j] += 1/3*t1[i]/tsum
        
for i in range(50):
    for j in range(4):
        c[i2[i] + j] += 1/4*t2[i]/tsum

for i in range(50):
    for j in range(5):
        c[i3[i] + j] += 1/5*t3[i]/tsum
        
c
np.sum(c)

(150,)
(3, 100, 50) (4, 100, 50) (5, 100, 50)
(50,) (50,) (50,)


1.0

In [444]:
raw_docs_test[ind[k]]

'If you sometimes like to go to the movies to have fun , Wasabi is a good place to start .'

In [445]:
wrds = []
sent = processed_docs_test[ind[k]]
wrd_prob = dict()

counter = 0
for i in sent:
    wrd_prob[i] = c[counter]
    counter += 1
    
import operator
# srt_list = sorted(wrd_prob.items())
sorted_x = sorted(wrd_prob.items(), key=operator.itemgetter(1), reverse=True)
sorted_x

[('wasabi', 0.23152487),
 ('good', 0.2180485),
 ('fun', 0.17998767),
 ('movi', 0.15142322),
 ('place', 0.12892881),
 ('go', 0.056666099),
 ('start', 0.020311475),
 ('like', 0.0084058866),
 ('sometim', 0.0033888198),
 ('if', 0.001314683)]

In [447]:
c

array([ 0.00131468,  0.00338882,  0.00840589,  0.0566661 ,  0.15142322,
        0.17998767,  0.23152487,  0.2180485 ,  0.12892881,  0.02031147,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ], dtype=float32)